# Motorcycle Night Ride
- Motorcycle Night Ride (Semantic Segmentation) 데이터셋 [링크](https://www.kaggle.com/datasets/sadhliroomyprime/motorcycle-night-ride-semantic-segmentation/)
- U-Net 모델 + a

## 목차


## 전처리